# Sears' Gust Frequency Response with SHARPy

This example script will illustrate the process for obtaining the frequency response of a very large aspect ratio flat plate subject to a sinusoidal gust using SHARPy. The results will be compared to the closed for solution for a 2D airfoil developed by Sears (1).

Given the large system, we will also turn to Krylov methods for model reduction, to efficiently compute the frequency response.

(1) - SHARPy is inherently a 3D solver and 2D solutions are approximated by very large aspect ratio wings that require high discretisations in order to achieve convergence.

In [1]:
# Packages:
import os
import sys
sys.path.append('~/code/sharpy/')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sc

import cases.templates.flying_wings as wings
import sharpy.sharpy_main
import sharpy.linear.src.lin_aeroelastic as linaeroela
import sharpy.linear.src.linuvlm as linuvlm
import sharpy.linear.src.libsparse as libsp
import sharpy.linear.src.libss as libss
import sharpy.rom.krylovreducedordermodel as krylovrom
import sharpy.rom.frequencyresponseplot as freqplot
import sharpy.utils.analytical as analytical

def save_variables(file_name, vars, var_title):
    fid = open(file_name, 'w')

    title_line = len(var_title)*'%s,' % var_title

    fid.write(title_line+'\n')

    for elem in range(vars[0].shape[0]):
        # var_line = len(var_title)*'%8f\t' % tuple(var_title[elem, :])
        vars_in_line = []
        vars_in_line.append([vars[i][elem] for i in range(len(var_title))])
        # print(vars_in_line[0])
        var_line = ''.join('%f,' % item for item in vars_in_line[0])
        # print(var_line)
        fid.write(var_line+'\n')

    fid.close()

## Set up the case parameters

Discretisation needs to be high - caution on resources

In [2]:
# Discretisation
M = 16
N = 80 # Warning. Can take a LOOONG time!
MstarFact = 10
nsurf = 1
rho = 1.225

# Flight Conditions
u_inf = 5
alpha_deg = 0
main_ea = 0.0
AR = 100

# Linear settings
remove_predictor = False
use_sparse = False
integration_order = 2

# ROM Settings
algorithm = 'dual_rational_arnoldi'
frequency_continuous_k = np.array([0.0])
krylov_r = 20

# Case Admin
case_route = os.path.abspath('.')
results_folder = case_route + '/res/'
fig_folder = case_route + '/figs/'
os.system('mkdir -p %s' % results_folder)
os.system('mkdir -p %s' % fig_folder)
case_name = 'sears_uinf%04d_AR%02d_M%dN%dMs%d_KR%d' % (u_inf, AR, M, N, MstarFact, krylov_r)

## Create Wing

Using SHARPy's templates

In [3]:
# Wing model
ws = wings.Goland(M=M,
                         N=N,
                         Mstar_fact=MstarFact,
                         n_surfaces=nsurf,
                         u_inf=u_inf,
                         rho = rho,
                         alpha=alpha_deg,
                         aspect_ratio=AR,
                         route=results_folder,
                         case_name=case_name)

ws.main_ea = main_ea
ws.clean_test_files()
ws.update_derived_params()
ws.generate_fem_file()
ws.generate_aero_file()

# Solution settings

ws.set_default_config_dict()
ws.config['SHARPy']['flow'] = ['BeamLoader', 'AerogridLoader', 'Modal', 'StaticUvlm', 'BeamPlot','AerogridPlot']

ws.config['LinearUvlm'] = {'dt': ws.dt,
                           'integr_order': integration_order,
                           'density': ws.rho,
                           'remove_predictor': remove_predictor,
                           'use_sparse': use_sparse,
                           'ScalingDict': {'length': 1.,
                                           'speed': 1.,
                                           'density': 1.}}
ws.config['Modal']['NumLambda'] = 40
ws.config['Modal']['keep_linear_matrices'] = 'on'
ws.config['Modal']['use_undamped_modes'] = True
ws.config.write()

## Nonlinear reference solution

In [ ]:
data = sharpy.sharpy_main.main(['...', results_folder + case_name + '.solver.txt'])
tsaero = data.aero.timestep_info[-1]
tsstruct = data.structure.timestep_info[-1]

/home/ng213/anaconda3/envs/sharpy_env/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/ng213/code/sharpy/sharpy/solvers/modal.py:288: RuntimeWarning: invalid value encountered in sqrt
  freq_natural = np.sqrt(eigenvalues)


## Linearised UVLM System

In [ ]:
# Linearisation parameters
dt = ws.dt
tsaero.rho = ws.rho
scaling_factors = {'length': 1,#0.5*ws.c_ref,
                   'speed': 1,#u_inf,
                   'density': 1}#ws.rho}

# Linearise UVLM
aeroelastic_system = linaeroela.LinAeroEla(data)
uvlm = aeroelastic_system.linuvlm
uvlm.assemble_ss()
aeroelastic_system.get_gebm2uvlm_gains()

# Remove lattice coordinates and velocities from the inputs to the system
uvlm.SS.B = libsp.csc_matrix(uvlm.SS.B[:, -uvlm.Kzeta:])
uvlm.SS.D = libsp.csc_matrix(uvlm.SS.D[:, -uvlm.Kzeta:])

## Gust Generator

We would like to define the gust at a particular location, for instance the leading edge. Therefore, that information must propagate downstream at the freestream velocity to reach the remainder of the airfoil.

If the gust velocity is orthogonal to the freestream, as is the case for this gust, a simple discrete time linear time invariant system can be created, which will receive the input at the leading edge and output the corresponding velocity at the chordwise wing nodes downstream.

This output will present the gust velocity at a single location along the span, so a gain will be necessary to map that information uniformly across the whole span.

In [ ]:
A_gust = np.zeros((M+1, M+1))
A_gust[1:, :-1] = np.eye(M)
B_gust = np.zeros((M+1, ))
B_gust[0] = 1
C_gust = np.eye(M+1)
D_gust = np.zeros_like(B_gust)
ss_gust = libss.ss(A_gust, B_gust, C_gust, D_gust, dt=ws.dt)

In [ ]:
# Gain to get uz at single chordwise position across entire span
K_lattice_gust = np.zeros((uvlm.SS.inputs, ss_gust.outputs))
for i in range(M+1):
    K_lattice_gust[i*(N+1):(i+1)*(N+1), i] = np.ones((N+1,))

# Add gain to gust generator
ss_gust.addGain(K_lattice_gust, where='out')

## UVLM Output

The linear UVLM system outputs the forces and moments at the lattice coordinates in the inertial frame. We must include a gain to condense them into a single vertical component in the A frame.

First, the lattice forces and moments are mapped onto the beam by means of the Kforces matrix. Thence, the nodal forces and moments are converted into a force coefficient.

In [ ]:
# UVLM - output: obtain vertical force
uvlm.SS.addGain(aeroelastic_system.Kforces, where='out')

K_Fz = np.zeros((1,aeroelastic_system.Kforces.shape[0]))
# Output - Vertical force coefficient

qS = 0.5 * ws.rho * u_inf ** 2 * ws.wing_span * ws.c_ref

wdof = 0
for node in range(data.structure.num_node):

    node_bc = data.structure.boundary_conditions[node]
    if node_bc != 1:
        node_ndof = 6
        vertical_force_index = np.array([0, 0, 1, 0, 0, 0]) / qS
        K_Fz[:, wdof: wdof + node_ndof] = vertical_force_index
    else:
        node_ndof = 0

    wdof += node_ndof

uvlm.SS.addGain(K_Fz, where='out')

## Combine the Gust and the UVLM systems

We now have a single input single output system with the following steps in between:

u_gust -> GUST -> gust along single chord -> K_lattice_gust -> gust velocity across span -> UVLM -> lattice forces -> Kforces -> nodal forces -> K_Fz -> vertical force coefficient

In [ ]:
sears_ss = libss.series(ss_gust, uvlm.SS)

## Model Order Reduction using Krylov Methods

The resulting SISO system has a very large number of states (~50^4) if convergence to the 2D result is desired and would normally require significant memory and time to compute the frequency response. Therefore, a ROM of the system is created using Krylov methods, based on the Arnoldi iteration around a single expansion point at zero-frequency, also known as a Pade approximation.

In [ ]:
rom = krylovrom.KrylovReducedOrderModel()
rom.initialise(data, sears_ss)
frequency_continuous_w = 2 * u_inf * frequency_continuous_k / ws.c_ref
frequency_dt = np.exp(frequency_continuous_k*dt)
rom.run(algorithm, krylov_r, frequency_dt)

## Frequency Analysis

In [ ]:
ds = 2. / M
fs = 1. / ds
fn = fs / 2.
ks = 2. * np.pi * fs
kn = 2. * np.pi * fn
Nk = 151
kv = np.linspace(0.01, 3, Nk)
wv = 2. * u_inf / ws.c_ref * kv

In [ ]:
Y_analytical = analytical.sears_fun(kv)
Y_freq_resp_rom = libss.freqresp(rom.ssrom, wv)

### Kussner 





In [ ]:
sc_kussner_A = sears_ss.A
sc_kussner_B = sears_ss.B
sc_kussner_C = sears_ss.C
sc_kussner_D = sears_ss.D

dlti_kussner = sc.signal.dlti(sc_kussner_A, sc_kussner_B, sc_kussner_C, sc_kussner_D, dt=dt)

Nsteps = 1000
t_dom = np.linspace(0, dt*Nsteps, Nsteps+1)

out = sc.signal.dlti(dlti_kussner, t_dom)

## Save results

Given the significant computational time to carry out the above, it is best to save the resulting frequency data to postprocess in a different routine.

In [ ]:
save_variables(results_folder + 'freq_data_' + case_name + '.csv', [wv, kv, Y_analytical.real, Y_analytical.imag, Y_freq_resp_rom[0,0,:].real, Y_freq_resp_rom[0,0,:].imag],
('wv', 'kv', 'Y_sears_r', 'Y_sears_i', 'Y_ROM_r', 'Y_ROM_i'))